In [1]:
import re
import string
from sklearn.model_selection import train_test_split
from nltk.util import ngrams,everygrams
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.lm import Vocabulary, MLE
# import pandas as pd
from numpy.random import multinomial as multi
from random import randrange, randint
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import Embedding
from pickle import load
from pickle import dump
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# from keras.layers import LSTM

Using TensorFlow backend.
/home/shaurya/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shaurya/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shaurya/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shaurya/anaconda3/lib/python3.7/site-packages/t

## Read, Parse and Divide Trump speech data

In [41]:
file=open("trumpspeeches.txt",'r')
text=file.read()

In [42]:
list_sent=list(sent_tokenize(text))

In [49]:
for sent in list_sent:
    sent=sent.replace('\n'," ")
    sent=sent.replace('-'," ")
    sent=sent.translate(str.maketrans('', '', string.punctuation))
    sent=sent.replace("’","")
    sent=sent.replace("–","")
    sent=re.sub(r"\d",'',sent)
    sent=sent.replace("SPEECH",'')
    sent=sent.replace('  '," ")
    sent=sent.lower()
    list_sent.pop(0)
    list_sent.append(sent)

In [50]:
list_sent[0]

' thank you so much'

In [51]:
text_train, text_test=train_test_split(list_sent,test_size=0.2,train_size=0.8, random_state=42)
print(len(text_train))
print(len(text_test))

13194
3299


In [52]:
text_train

['or stupid',
 'not going to happen anymore',
 'believe me',
 'he signed that',
 'our infrastructure is a disaster',
 'but we have so many people like that',
 'i mean youre in the crazy world of politics',
 'its true',
 'i wonder if time magazine had that one to do over again what theyd say',
 'we have leadership again no leadership respects us',
 'we mourn as one people for our nations loss and pledge our support to any and all who need it',
 'i love all these people',
 'we gave them iraq that theyve been after for years',
 'we gave iran iraq',
 'and i was so nice',
 'they couldnt care less',
 'the worst',
 'you talk about a great franchise',
 'and you have a truly great quarterback',
 'very important',
 'right',
 'yeah go ahead finish it',
 'let me tell you something',
 'the judge has been very unfair',
 'politicians are all talk no action',
 'now hes saying well never pay for the wall',
 'nobody even knows why',
 'i can settle that case',
 'these are the stats',
 'we cant have it',


## Q1

In [61]:
def gettoken(sent_tokens,if_sent=False):
    tokens=[]
    if(if_sent==False):
        for i in range(len(sent_tokens)):
            temp=word_tokenize(sent_tokens[i])
            tokens.append('<s>')
            tokens.extend(temp)
            tokens.append('</s>')
    else:
        temp=word_tokenize(sent_tokens)
        tokens.append('<s>')
        tokens.extend(temp)
        tokens.append('</s>')
    return tokens

In [62]:
vocab=[]
train_tokens=gettoken(text_train)
vocab=np.unique(train_tokens)
output=[]
for i in range(len(text_test)):
    output.append(gettoken(text_test[i],if_sent=True))
# vocab=np.unique(np.concatenate((vocab,np.array(token1)), axis=None))
# print(len(vocab))

In [63]:
train_tokens

['<s>',
 'or',
 'stupid',
 '</s>',
 '<s>',
 'not',
 'going',
 'to',
 'happen',
 'anymore',
 '</s>',
 '<s>',
 'believe',
 'me',
 '</s>',
 '<s>',
 'he',
 'signed',
 'that',
 '</s>',
 '<s>',
 'our',
 'infrastructure',
 'is',
 'a',
 'disaster',
 '</s>',
 '<s>',
 'but',
 'we',
 'have',
 'so',
 'many',
 'people',
 'like',
 'that',
 '</s>',
 '<s>',
 'i',
 'mean',
 'youre',
 'in',
 'the',
 'crazy',
 'world',
 'of',
 'politics',
 '</s>',
 '<s>',
 'its',
 'true',
 '</s>',
 '<s>',
 'i',
 'wonder',
 'if',
 'time',
 'magazine',
 'had',
 'that',
 'one',
 'to',
 'do',
 'over',
 'again',
 'what',
 'theyd',
 'say',
 '</s>',
 '<s>',
 'we',
 'have',
 'leadership',
 'again',
 'no',
 'leadership',
 'respects',
 'us',
 '</s>',
 '<s>',
 'we',
 'mourn',
 'as',
 'one',
 'people',
 'for',
 'our',
 'nations',
 'loss',
 'and',
 'pledge',
 'our',
 'support',
 'to',
 'any',
 'and',
 'all',
 'who',
 'need',
 'it',
 '</s>',
 '<s>',
 'i',
 'love',
 'all',
 'these',
 'people',
 '</s>',
 '<s>',
 'we',
 'gave',
 'them',


In [64]:
def getgram(n):
    grams = list(everygrams(train_tokens,max_len=n))
    mle = MLE(n)
    mle.fit([grams], vocabulary_text=vocab)
    return mle
for i in range(1,5):
    print("No of ",i,"-gram Theoretically possible: ",len(vocab)**i)
    print("No of ",i,"-gram Practically: ",len(set(ngrams(train_tokens,i))))
unigram=getgram(1)
bigram=getgram(2)
trigram=getgram(3)
quadgram=getgram(4)
# print("lo")

No of  1 -gram Theoretically possible:  4364
No of  1 -gram Practically:  4364
No of  2 -gram Theoretically possible:  19044496
No of  2 -gram Practically:  29209
No of  3 -gram Theoretically possible:  83110180544
No of  3 -gram Practically:  53891
No of  4 -gram Theoretically possible:  362692827894016
No of  4 -gram Practically:  74341


In [65]:
unigram

In [66]:
def generate(n,model):
    start="<s>"
    gen_sent=[start]
    if(n==1):
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i) for i in vocab]))
            gen_sent.append(vocab[index])
            start=vocab[index]
    elif(n==2):
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i,[start]) for i in vocab]))
            gen_sent.append(vocab[index])
            start=vocab[index]
    elif(n==3):
        gen=[start]
        index=np.argmax(multi(1,[model.score(i,[start]) for i in vocab]))
        gen_sent.append(vocab[index])
        gen.append(vocab[index])
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i,gen) for i in vocab]))
            gen_sent.append(vocab[index])
            start=vocab[index]
            gen=gen[1:]
    elif(n==4):
        gen=[start]
        index=np.argmax(multi(1,[model.score(i,[start]) for i in vocab]))
        gen_sent.append(vocab[index])
        gen.append(vocab[index])
        index=np.argmax(multi(1,[model.score(i,gen) for i in vocab]))
        gen_sent.append(vocab[index])
        gen.append(vocab[index])
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i,gen) for i in vocab]))
            gen_sent.append(vocab[index])
            start=vocab[index]
            gen=gen[1:]
    return ' '.join(gen_sent)
print("Unigram:")
for i in range(10):
    print(generate(1,unigram))
print("\nBigram:")
for i in range(10):
    print(generate(2,bigram))
print("\nTrigram:")
for i in range(10):
    print(generate(3,trigram))
print("\nQuadgram:")
for i in range(10):
    print(generate(4,quadgram))

Unigram:
<s> to many is i of lousy and christmas costs </s>
<s> the tell for </s>
<s> were them leave killed folks will idea country of it a tonight look </s>
<s> dealing stability a dishonesty iraq toward have signed the to because </s>
<s> zone in </s>
<s> were </s>
<s> </s>
<s> they many the </s>
<s> lets him for enemy to that trump great we is </s>
<s> china used rush </s>

Bigram:
<s> everybody just want clean crystal water shed do you from the states only on trade deals because this i speak the a nation for us </s>
<s> and now theyre leaving our allies are you think beating us cars are totally over here because later because you build a security </s>
<s> do it but the big — he refuses to study new york times two or one of the bottom line they left </s>
<s> they would be elected president for both parties — first again </s>
<s> they are not leading by the term </s>
<s> we have so many people </s>
<s> dont think — fox las vegas percent and they drive cars right now without a long t

In [67]:
from random import randrange
tes=[randrange(0,len(output),1) for j in range(10)]
def perplexity(models):
    for i in tes:
        print("PP of ",' '.join(output[i]))
        for j in range(len(models)):
            mle=models[j]
            try:
                print(j+1," Gram ",round(mle.perplexity(ngrams(output[i],j+1)),2))
            except:
                print(j+1," Gram ","inf")
models=[unigram,bigram,trigram,quadgram]
perplexity(models)

PP of  <s> and by the way when obamacare kicks in which is really kicking in in you want to see bad deficits </s>
1  Gram  inf
2  Gram  inf
3  Gram  inf
4  Gram  inf
PP of  <s> i said well what do you think </s>
1  Gram  98.52
2  Gram  inf
3  Gram  inf
4  Gram  inf
PP of  <s> sharon did you take a lot of incoming </s>
1  Gram  333.03
2  Gram  23.22
3  Gram  3.82
4  Gram  2.69
PP of  <s> you know when ive got a big audience </s>
1  Gram  193.7
2  Gram  23.52
3  Gram  5.03
4  Gram  2.12
PP of  <s> and they said it started because of lyndon johnson </s>
1  Gram  308.26
2  Gram  31.93
3  Gram  6.25
4  Gram  2.06
PP of  <s> i know that my life will continue </s>
1  Gram  178.36
2  Gram  27.12
3  Gram  7.14
4  Gram  1.42
PP of  <s> remember that </s>
1  Gram  66.35
2  Gram  17.34
3  Gram  2.46
4  Gram  1.17
PP of  <s> very liberal paper but thats okay </s>
1  Gram  613.32
2  Gram  24.2
3  Gram  2.01
4  Gram  1.0
PP of  <s> but their leaders are much smarter than our leaders and we cant susta

# Comment on readability
Quadgrams are more readable than tri, which are more readable than bi which are more readable than uni


## Q2

In [68]:
n=25

In [77]:
train_tokens=[]
for i in range(len(text_train)):
    train_tokens.extend(word_tokenize(text_train[i]))

In [78]:
length = n + 1
train_sequences = list()
for i in range(length, len(train_tokens)):
    seq = train_tokens[i-length:i]
    line = ' '.join(seq)
    train_sequences.append(line)
print('Total Sequences: %d' % len(train_sequences))
train_seq_copy=train_sequences[::]

Total Sequences: 132527


In [79]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_sequences)
sequences = tokenizer.texts_to_sequences(train_sequences)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
 
# separate into input and output
lines = np.array(sequences)
X, y = lines[:,:-1], lines[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [107]:
# define model
def lstm_mine():
    model = Sequential()
    model.add(Embedding(vocab_size, n, input_length=seq_length))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100))
#     model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    print(model.summary())
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit model
    model.fit(X, y, batch_size=128, epochs=10)
    model.save('LSTMmodel.h5')
    dump(tokenizer, open('LSTMtokenizer.pkl', 'wb'))

def VanillaRNN():
    model = Sequential()
    model.add(Embedding(vocab_size, n, input_length=seq_length))
    model.add(SimpleRNN(100, return_sequences=True))
    model.add(SimpleRNN(100))
#     model.add(Dropout(0.1))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    print(model.summary())
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, y, batch_size=128, epochs=10)
    model.save('RNNmodel.h5')
    dump(tokenizer, open('RNNtokenizer.pkl', 'wb'))
lstm=lstm_mine()
vanilla=VanillaRNN()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 25, 25)            109075    
_________________________________________________________________
lstm_19 (LSTM)               (None, 25, 100)           50400     
_________________________________________________________________
lstm_20 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_31 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_32 (Dense)             (None, 4363)              440663    
Total params: 690,638
Trainable params: 690,638
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
132527/132527 [==============================] - 72s 543us/step - loss: 6.0658 - acc: 0.0472
Epoch 2/10
132527/132527 [=

In [112]:
def generate(model, tokenizer, seq_length, seed_text, n_words):
#     result = list()
    in_text = seed_text
    for i in range(n_words):
        seq_enc = tokenizer.texts_to_sequences([in_text])[0]
        seq_enc = pad_sequences([seq_enc], maxlen=seq_length, truncating='pre')
        yhat = model.predict_classes(seq_enc, verbose=0)
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        in_text += ' ' + out_word
#         result.append(out_word)
    return in_text
gen_sequences=train_seq_copy
modelLstm = load_model('LSTMmodel.h5')
tokenizerLstm = load(open('LSTMtokenizer.pkl', 'rb'))
modelRnn = load_model('RNNmodel.h5')
tokenizerRnn = load(open('RNNtokenizer.pkl', 'rb'))
print("LSTM")
for i in range(5):
    seed_text = gen_sequences[randint(0,len(gen_sequences))]
    print("Input"+"\n"+seed_text)
    generatedLSTM = generate(modelLstm, tokenizerLstm, len(gen_sequences[0].split()) - 1, seed_text,50)
    print("Output"+"\n"+generatedLSTM+"\n")
print()
print()
print("RNN")
for i in range(5):
    seed_text = gen_sequences[randint(0,len(gen_sequences))]
    print("Input"+"\n"+seed_text)
    generatedRNN = generate(modelRnn, tokenizerRnn, len(gen_sequences[0].split()) - 1, seed_text, 50)
    print("Output"+"\n"+generatedRNN+"\n")

LSTM
Input
no leader were never going to have to use it i mean its just total control youre going to have problems i mean you will see
Output
no leader were never going to have to use it i mean its just total control youre going to have problems i mean you will see was a great come and i for want to its a great come and i for want to its a great come and i for want to its a great come and i for want to its a great come and i for want to its a great come and

Input
the best thing i could do is say but i cant destroy a man so we sent them a its a scam super pacs a scam
Output
the best thing i could do is say but i cant destroy a man so we sent them a its a scam super pacs a scam i for know i for know i for know i for know i for know i for know i for know i for know i for know i for know i for know i for know i for know i for know i for know i for know i for

Input
mean — but — but if you look at my calls i said dont go into iraq thats my own problem fifty thousand factories across americ

In [109]:
test_tokens=[]
for i in range(len(text_test)):
    test_tokens.extend(word_tokenize(text_test[i]))
test_sequences=[]
for i in range(length, len(output)):
    seq = test_tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    test_sequences.append(line)
tokenizer.fit_on_texts(test_sequences)
test_seq = tokenizer.texts_to_sequences(test_sequences)
test_seq = np.array(test_seq)
X_test, y_test = test_seq[:,:-1], test_seq[:,-1]
y_test = to_categorical(y_test,num_classes=vocab_size)

In [110]:
def perplexityNeural(model):
    score = model.evaluate(X_test, y_test,verbose=0)
    print('Test score:', score)
    print("Perplexity:", np.exp(score))

In [111]:
print("LSTM")
perplexitylLstm = load_model('LSTMmodel.h5')
perplexityNeural(modelLstm)
# modelLstm.evaluate()
print("Vanilla RNN")
modelRnn = load_model('RNNmodel.h5')
perplexityNeural(modelRnn)

LSTM
Test score: [7.858476206828723, 0.04613504430414011]
Perplexity: [2.58757444e+03 1.04721582e+00]
Vanilla RNN
Test score: [8.213411654458657, 0.044912923925282795]
Perplexity: [3.69011040e+03 1.04593678e+00]


## Comment on Readabiliy
They are readable. But word-level LSTM and RNN can only give good answers if trained fo 

## Comparision of Neural and Classical

The Classical approach is better when the test data is same as train data but this doesn't happen here hence the perplexity becomes inf in many cases. But Neural approach is better for real-world approach, because it can generalize to test data when its not same as train data.